In [1]:
input_file = "15_input.txt"

In [2]:
from tqdm import tqdm
from bisect import bisect_left, bisect_right

In [3]:
sensors = []

with open(input_file) as f:
    for line in f:
        line = line.rstrip().split()
        sensor = [int(line[i][2:].rstrip(",:")) for i in [2, 3]]
        beacon = [int(line[i][2:].rstrip(",:")) for i in [8, 9]]
        sensors.append([sensor, beacon])

In [4]:
def distance(p1, p2):
    # manhattan distance between two points
    return sum(abs(c1-c2) for c1, c2 in zip(p1, p2))


class LocationSystem:
    def __init__(self, sensorlist):
        self.sensors = [o[0] for o in sensorlist]
        self.beacons = [o[1] for o in sensorlist]
        self.unique_beacons = list(list(b) for b in set(tuple(b) for b in self.beacons))
        self.distances = [distance(sensor, beacon) for sensor, beacon in zip(self.sensors, self.beacons)]
        all_x = [c[0] for c in self.sensors + self.beacons]
        all_y = [c[1] for c in self.sensors + self.beacons]
        self.xlim = min(all_x), max(all_x)
        self.ylim = min(all_y), max(all_y)
        self.width = 1 + self.xlim[1] - self.xlim[0]
        self.height = 1 + self.ylim[1] - self.ylim[0]
    
    
    def print_grid(self, chars='.SB#', excluded=False, restrict=None):
        grid = [[chars[0]] * (self.width) for _ in range(self.height)]
        if excluded:
            for (x, y), d in zip(self.sensors, self.distances):
                ymin = max(self.ylim[0], y - d) - self.ylim[0]
                ymax = min(self.ylim[1], y + d) - self.ylim[0]
                for ry in range(ymin, ymax + 1):
                    dy = abs(y - ry)
                    dx = d - dy
                    xmin = max(self.xlim[0], x - dx) - self.xlim[0]
                    xmax = min(self.xlim[1], x + dx) - self.xlim[0]
                    for rx in range(xmin, xmax + 1):
                        grid[ry][rx] = chars[3]
                    #grid[ry][xmin : xmax + 1] = chars[3]
        for x, y in self.sensors:
            grid[y - self.ylim[0]][x - self.xlim[0]] = chars[1]
        for x, y in self.unique_beacons:
            grid[y - self.ylim[0]][x - self.xlim[0]] = chars[2]
        print("    " + ''.join('x' if x % 5 == 0 else ' ' for x in range(self.xlim[0], self.ylim[1]+1)))
        for k, row in enumerate(grid):
            print(f"{k+self.ylim[0]:3}", ''.join(row))
    
    @staticmethod
    def excluded_row(row, pos, dmax):
        dy = abs(pos[1] - row)
        dx = dmax - dy
        if dx < 0:
            return 0, (None, None)
        xmin = pos[0] - dx
        xmax = pos[0] + dx
        return 2 * dx + 1, (xmin, xmax)
    
    
    def count_excluded(self, row):
        left_edges = []
        right_edges = []
        for pos, dmax in zip(self.sensors, self.distances):
            w, (xmin, xmax) = self.excluded_row(row, pos, dmax)
            if w > 0:
                kl = bisect_left(left_edges, xmin)
                kr = bisect_right(right_edges, xmax)
                if kl == 0 or xmin > right_edges[kl - 1]:
                    new_left_edges = left_edges[:kl] + [xmin]
                    new_right_edges = right_edges[:kl]
                else:
                    new_left_edges = left_edges[:kl]
                    new_right_edges = right_edges[:kl-1]
                if kr == len(right_edges) or xmax < left_edges[kr]:
                    new_left_edges += left_edges[kr:]
                    new_right_edges += [xmax] + right_edges[kr:]
                else:
                    new_left_edges += left_edges[kr+1:]
                    new_right_edges += right_edges[kr:]
                left_edges = new_left_edges
                right_edges = new_right_edges
        excluded_legth = sum(1 + r - l for l, r in zip(left_edges, right_edges))
        n_beacons = sum(1 for b in self.unique_beacons if b[1] == row)
        return excluded_legth - n_beacons
    
    
    def allowed_in_row(self, row, rmin, rmax):
        left_edges = []
        right_edges = []
        for pos, dmax in zip(self.sensors, self.distances):
            w, (xmin, xmax) = self.excluded_row(row, pos, dmax)
            if w > 0:
                xmin = max(xmin, rmin)
                xmax = min(xmax, rmax)
                kl = bisect_left(left_edges, xmin)
                kr = bisect_right(right_edges, xmax)
                if kl == 0 or xmin > right_edges[kl - 1]:
                    new_left_edges = left_edges[:kl] + [xmin]
                    new_right_edges = right_edges[:kl]
                else:
                    new_left_edges = left_edges[:kl]
                    new_right_edges = right_edges[:kl-1]
                if kr == len(right_edges) or xmax < left_edges[kr]:
                    new_left_edges += left_edges[kr:]
                    new_right_edges += [xmax] + right_edges[kr:]
                else:
                    new_left_edges += left_edges[kr+1:]
                    new_right_edges += right_edges[kr:]
                left_edges = new_left_edges
                right_edges = new_right_edges
        if len(left_edges) == 2 and left_edges[1] == right_edges[0] + 2:
            return left_edges[1] - 1
        
        
    def find_allowed(self, xlim, ylim):
        for row in tqdm(range(ylim[0], ylim[1] + 1)):
            found = self.allowed_in_row(row, xlim[0], xlim[1])
            if found is not None:
                return found, row
        return None, row
            
    
    def tuning_freq(self, xlim=[0, 4_000_000], ylim=[0, 4_000_000]):
        x, y = self.find_allowed(xlim, ylim)
        if x is not None:
            return x * 4_000_000 + y
        raise ValueError("No allowed position was found")
    
    
    def check_allowed(self, x, y):
        for sensor, d in zip(self.sensors, self.distances):
            if distance([x, y], sensor) <= d:
                return False
        return True
    
    
    def slow_find(self, xlim, ylim):
        # runs in ~1-2 years
        for x in tqdm(range(xlim[0], xlim[1] + 1), desc="x"):
            for y in tqdm(range(ylim[0], ylim[1] + 1), desc="    y", miniters=(ylim[1]-ylim[0])//2):
                if self.check_allowed(x, y):
                    return x, y

In [5]:
cave = LocationSystem(sensors)

#part 1
cave.count_excluded(2_000_000)

4665948

In [6]:
cave.tuning_freq()

 67%|████████████████████████████████████████████████                        | 2671045/4000001 [00:27<00:13, 98133.06it/s]


13543690671045